In [1]:
pwd

'/home/andrew/side_projects/pysyft-ak/PySyft/examples/experimental'

In [2]:
cd ~/side_projects/pysyft-ak/PySyft/

/home/andrew/side_projects/pysyft-ak/PySyft


In [3]:
import torch as th
import syft as sy
# sy.create_sandbox(globals())
hook = sy.TorchHook(th)

## Simplified problem

- each worker starts with a number that they hold locally
- at the end, each worker has the sum of all numbers

In [4]:
a,b,c = th.tensor(1), th.tensor(2), th.tensor(3)

In [15]:
alice, bob, carol = (
    sy.VirtualWorker(hook, 'alice'),
    sy.VirtualWorker(hook, 'bob'),
    sy.VirtualWorker(hook, 'carol')
)
workers = [alice, bob, carol]
dan = sy.VirtualWorker(hook, 'dan')

# Start with these pointer to tensors that "live on workers already"
a_ptr = a.send(alice)
b_ptr = b.send(bob)
c_ptr = c.send(carol)

In [16]:
# Workers share their data amongst themselves.
# Now each owns shares of each's number, as well as a pointer to the shared data for its number

a_ptr_shared = a_ptr.share(*workers, crypto_provider=dan)
b_ptr_shared = b_ptr.share(*workers, crypto_provider=dan)
c_ptr_shared = c_ptr.share(*workers, crypto_provider=dan)

In [19]:
print(a_ptr_shared)

(Wrapper)>[PointerTensor | me:40742811351 -> alice:59863524400]
None


In [8]:
# Another party (could be one of the workers, I guess) 
# accumulates pointers to shared tensors
a_shared = a_ptr_shared.get()
b_shared = b_ptr_shared.get()
c_shared = c_ptr_shared.get()

In [9]:
# Pointers to shared tensors are added
total_shared = a_shared + b_shared + c_shared

In [10]:
# result is reconstructed on each worker!
reconstructed = total_shared.child.reconstruct()

## Same thing, but leveraging multipointer tensors

In [91]:
a,b,c = th.tensor([[1.1,1.1]]), th.tensor([[2.2, 2.2]]), th.tensor([[3.3, 3.3]])

In [92]:
alice, bob, carol = (
    sy.VirtualWorker(hook, 'alice'),
    sy.VirtualWorker(hook, 'bob'),
    sy.VirtualWorker(hook, 'carol')
)
workers = [alice, bob, carol]
dan = sy.VirtualWorker(hook, 'dan')

# Start with these pointer to tensors that "live on workers already"
a_ptr = a.send(alice)
b_ptr = b.send(bob)
c_ptr = c.send(carol)

In [93]:
m = sy.MultiPointerTensor(children=[a_ptr, b_ptr, c_ptr])

In [94]:
m.child['alice']

(Wrapper)>[PointerTensor | me:49713571511 -> alice:85240384378]

In [88]:
ptrs_to_shared = []
for worker in workers:
    ptrs_to_shared.append(m.child[worker.id].share(*workers, crypto_provider=dan).get())

In [90]:
ptrs_to_shared[0].get()

tensor([[1, 1]])

In [75]:
summed = sum(ptrs_to_shared)

In [76]:
reconstructed = summed.child.reconstruct()

In [77]:
reconstructed.child['carol'].get()

tensor([[6, 6]])

In [95]:
x = th.tensor([1.1])

In [96]:
x.fix_precision()

(Wrapper)>FixedPrecisionTensor>tensor([1100])